# Collecting Tweets using Tweepy

In [2]:
# Load packages
import tweepy as tw #make sure to install v3.10
import pandas as pd
import numpy as np
# install preprocessor using pip install via Command Prompt/Anaconda prompt: pip install tweet-preprocessor
from preprocessor import api as p
#from preprocessor.api import clean, tokenize, parse #to clean tweets

In [3]:
# Credentials
access_token = "1311658729201377281-alEuRquK89jGbga7ckTsFhIlkRVPOY"
access_token_secret = "rpQowQaio69m3XMByxXEzUdo73WPnpaPuYTQpAFH9cjoa"
api_key = "E0RK2oHynrMIJ9YVrvuhIIjc6"
api_key_secret = "kHnfmtYn7J66MkR7bUE3BTnWsW64ZUDUIxtw9bThP5kmDjq0Vl"

In [4]:
# Authentication app
auth = tw.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth)

In [31]:
# Find by phrase or keywords within a specified time
search_words = '"lake erie" has:links'
#search_words = '"lake erie" -has:links' 
# we can not use -filter:links etc for premium API search
# use this link instead https://developer.twitter.com/en/docs/twitter-api/premium/rules-and-filtering/using-premium-operators
# it is equivalent to utah lake until:2020-12-20 since:2020-11-28 -filter:links
# we can't filter out retweets in the premium search

# advanced search in Twitter search box 
# utah lake (#utahlake) until:2020-12-03 since:2020-11-28 -filter:links -filter:retweets
# If I include hashtag in search, it wouldn't show the ones without hashtag
# May be we can have a separate search by #utahlake and then combine both the dataframes

# From date, to date, for premium API search the date should be in format yyyyMMddHHmm
year_from = int(input("Year from: "))
year_to = int(input("Year to: "))
date_since = f'{year_from}01010000' #from January 1st, 12:00AM UTC
#date_until = f'{year_to}12312359' #to December 31st, 11:59PM UTC
date_until = f'{year_to}12310300' #for 2020 3AM UTC

#date_since = f'{year}12250000' #test
#date_until = f'{year}12280000' #test

# Create a cursor object that iterates over the requested information
# had to create a dev environment named "SentimentAnalysis" in Twitter's Developer Portal
tweets = tw.Cursor(api.search_full_archive, environment_name = "SentimentAnalysis",
                   query=search_words,
                   fromDate=date_since, toDate=date_until).items()

# Parse the cursor object into rows of lists using list comprehension
#rows = [[tweet.user.location, tweet.text, tweet.created_at, tweet.coordinates] for tweet in tweets]

# Make a list to store each item return in a payload
rows = []

# Iterate through each object and get location, tweet, date, coordinates and place for each record
for tweet in tweets:
    
    # check if there is an 'extended_tweet' property
    if ('extended_tweet' in dir(tweet)) == True: 
        # if so, instead of 'text', get 'full_text'
        row = [tweet.user.location, tweet.extended_tweet['full_text'], tweet.created_at, tweet.coordinates, tweet.retweeted, tweet.place]
    else:
        row = [tweet.user.location, tweet.text, tweet.created_at, tweet.coordinates, tweet.retweeted, tweet.place]

    # Append each item to the list
    rows.append(row)

# Tweets starting with an RT mean that they are retweets 

# more on geo-referenced tweets: 
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/geo
# if required, look into user.derived.locations.geo
# it assigns lat/long based on user's profile and the geographic precision they set
# for example, if a user set their precision at the country-level, it will show center of US at lat/long
# so it is not too accurate and might be irrelevant to us
# https://developer.twitter.com/en/docs/twitter-api/enterprise/enrichments/overview/profile-geo

Year from: 2013
Year to: 2020


KeyboardInterrupt: 

In [32]:
# Create dataframe from the rows of lists
df_tweet = pd.DataFrame(data=rows, columns=['location', 'tweet', 'time', 'coordinates', 'retweeted', 'place'])

In [33]:
# Add ID column (because we removed user information)
unique_id = np.arange(len(df_tweet))
df_tweet['id'] = unique_id + 1
df_tweet = df_tweet[['id', 'location', 'tweet', 'time', 'coordinates', 'retweeted', 'place']]

In [34]:
df_tweet.head(50)

,id,location,tweet,time,coordinates,retweeted,place
0,1,"Ingersoll, Ontario",New video from Aug. 17th intercept of severe t...,2020-12-31 02:58:41,None,False,None
1,2,"Cleveland, OH",Iconic tree along Lake Erie comes down at Hunt...,2020-12-31 02:49:02,None,False,None
2,3,"Cleveland, Ohio",The 3News Lake Erie Nearshore Forecast... http...,2020-12-31 02:45:33,None,False,None
3,4,Northern Virginia,RT @Gabby_Hoffman: NEW VIDEO: Trophy Steelhead...,2020-12-31 02:44:01,None,False,None
4,5,"Ohio, USA",@grneyedmusiklvr We get some amazing sunrises ...,2020-12-31 02:40:12,None,False,None
5,6,None,RT @WIRED: When nasty storms pummel the Great ...,2020-12-31 02:27:11,None,False,None
6,7,None,RT @WIRED: When nasty storms pummel the Great ...,2020-12-31 01:58:38,None,False,None
7,8,"Denver, PA",Possibly one of the most iconic relics of the ...,2020-12-31 01:47:15,None,False,None
8,9,"California, USA",RT @Floatie_Flo: Received an offer to finish m...,2020-12-31 01:43:36,None,False,None
9,10,None,RT @HCChargers: What's better than a rivalry g...,2020-12-31 01:31:34,None,False,None


In [35]:
df_tweet.tail()

,id,location,tweet,time,coordinates,retweeted,place
74977,74978,Canberra,#NewYork state #homes covered with amazing #ic...,2020-03-04 21:30:32,None,False,None
74978,74979,Desubicada,RT @john_kucko: 19 Days Until Spring: Until t...,2020-03-04 21:27:07,None,False,None
74979,74980,Desubicada,RT @StormchaserUKEU: WOW... Absolutely fantast...,2020-03-04 21:26:57,None,False,None
74980,74981,United States,https://t.co/yoTW7fCezG,2020-03-04 21:23:13,None,False,None
74981,74982,"Congleton, Cheshire. UK","After extreme winds, homes along Lake Erie cov...",2020-03-04 21:13:58,None,False,None


In [36]:
df_tweet.tweet[1]

'Iconic tree along Lake Erie comes down at Huntington Beach in Bay Village https://t.co/DFMuyXv2om https://t.co/MW7k8kSIzw'

In [37]:
df_tweet.place[1]

# Cleaning tweets in the dataframe 

#### Lets clean tweets after labeling

In [38]:
# # Clean each tweet in our dataframe using the preprocessor module
# tweet_clean = []
# for tweet in df_tweet['tweet']:
#     cleaned = p.clean(tweet)
#     tweet_clean.append(cleaned)

In [39]:
# # Store as a new dataframe
# df_tweet_clean = df_tweet.copy()
# df_tweet_clean['tweet'] = tweet_clean

In [40]:
#df_tweet_clean.head()

# Save the dataframe as a CSV

In [41]:
# Checking current working directory
import os
os.getcwd()

'C:\\Users\\rapiduser\\Documents\\WaterQualityPerception\\Pierre\\WaterQualityPerceptionMP2020'

In [42]:
# Store as CSV to a relative path
#df_tweet.to_csv(f"..\\..\\Data\\lakeerie_{year_from}_{year_to}.csv")
df_tweet.to_csv(f"..\\..\\Data\\lakeerie_{year_from}_{year_to}links.csv")
#df_tweet_clean.to_csv(f"..\\..\\Data\\utahlake_{year}.csv")
#df_tweet_clean.to_csv("..\\..\\Data\\scraped_tweets_test.csv")